# Calculate Fréchet Inception Distance (FID) Score

In [1]:
import tensorflow as tf

import fid
import pathlib

### FID

The FID score was introduced by Heusel et al. (2017, see: https://arxiv.org/abs/1706.08500 ) in order to improve on the currently established inception score (IS) for the evaluation of image generation DL methods, specifically when applying GAN architectures. FID - in comparison to IS - has the ability to evaluate the quality of generated images by comparing a statistical distribution of a latent representation feature vector based on the InceptionV3 model with the same same statistical distribution of the original images.

FID achieves this by using the Fréchet distance between the two distributions representing the real and generated images as follows:

<div align="center">
    <img src="FID.PNG"></img><br>
    <i>source: https://jonathan-hui.medium.com/gan-how-to-measure-gan-performance-64b988c47732</i>
</div>
<br>
<br>

where:<br>
    x = real images<br>
    g = generated images<br>
    µx = mean of the multivariate Gaussian distribution representing the latent vector of real images<br>
    µg = mean of the multivariate Gaussian distribution representing the latent vector of generated images<br>
    Σx = covariance matrix for real images<br>
    Σg = covariance matrix for generated images<br>

Essentially, calculating a FID score for two sets of images requires the following steps:

1. Load <i>Inception V3</i> model
2. Modify <i>Inception V3</i> so that we discard the output layer (classification into image categories), only keeping a max pooling layer representing the latent space vector.
3. Calculate the <i>mean (µ)</i> and <i>variance (Σ)</i> based on the latent space vector of the real (training) images
4. Generate images (here using a GAN)
5. Calculate the <i>mean (µ)</i> and <i>variance (Σ)</i> based on the latent space vector of the generated ("fake") images
6. Calculate FID based on Fréchet Distance between the two statistical distributions

Luckily the implementation provided by the paper's authors takes care of step 1. and 2., so here we start with step 3.


### Calculate Training Statistics

In [2]:
# implementation by authors preloads all images to RAM to calculate real distribution parameters. with 700'000 images this is not possible.
# I may have to custom implement a FID network based on Inception V3 by removing the last layer (see: https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/)
inception_path = fid.check_or_download_inception(None)
fid.create_inception_graph(inception_path) 


# load training dataset (full)
IMAGE_SIZE = (1024, 1024)
BATCH_SIZE = 64

data_dir = pathlib.Path('/data/input/crops')
imgs = list(data_dir.glob('*.png'))

dataset_train = tf.keras.preprocessing.image_dataset_from_directory(  '/data/input/',
                                                                      image_size=IMAGE_SIZE, 
                                                                      batch_size=BATCH_SIZE, 
                                                                      labels=[1.] * len(imgs), # setting all labels to 1 (for 'real')
                                                                      #label_mode=None, # yields float32 type labels
                                                                      seed=42
                                                                    )



Found 726300 files belonging to 1 classes.


In [ ]:
len(dataset_train)
for i in dataset_train.take(1):
    print(i)
    if i == 1:
        break

In [3]:
fid.create_inception_graph(inception_path)  # load the graph into the current TF graph
with tf.compat.v1.Session() as sess:
    sess.run(tf.global_variables_initializer())
    mu_gen, sigma_gen = fid.calculate_activation_statistics(dataset_train, sess, batch_size=BATCH_SIZE)

AttributeError: module 'tensorflow' has no attribute 'Session'